In [3]:
# Parameters and imports
from pathlib import Path
import json
import sqlite3
from collections import defaultdict

import polars as pl
import plotly.graph_objects as go
import plotly.express as px

# Paths (robust for notebooks)
repo_cwd = Path.cwd()
solver_dir = repo_cwd / "deterministic_vrp_solver"
if not (solver_dir / "test_decomposed_system_solution.json").exists():
    # If notebook is already in deterministic_vrp_solver or paths differ
    solver_dir = repo_cwd

# Solution and data paths
solution_path = solver_dir / "test_decomposed_system_solution.json"
if solver_dir.name == "deterministic_vrp_solver":
    orders_path = solver_dir.parent / "ml_ozon_logistic" / "ml_ozon_logistic_dataSetOrders.json"
else:
    orders_path = repo_cwd / "ml_ozon_logistic" / "ml_ozon_logistic_dataSetOrders.json"
ports_db_path = solver_dir / "ports_database.sqlite"

# Visualization options
show_ports = True
line_opacity = 0.45
order_marker_size = 5
port_marker_size = 8



In [4]:
# Load data
# Solution
with open(solution_path, "r", encoding="utf-8") as f:
    solution = json.load(f)
routes = solution.get("routes", [])

# Orders (flatten)
orders_df = (pl.read_json(str(orders_path))
    .explode("Orders")
    .unnest("Orders")
    .select(["ID", "MpId", "Lat", "Long"]) 
)
# Build order -> (lat, lon, mp)
order_to_geo = {row[0]: (row[2], row[3], row[1]) for row in orders_df.iter_rows()}

# Ports DB: polygon ports list and optional coordinates by ID (if present)
conn = sqlite3.connect(str(ports_db_path))
cur = conn.cursor()
# polygon_ports: polygon_id, port_id
cur.execute("SELECT polygon_id, port_id FROM polygon_ports")
rows = cur.fetchall()
polygon_to_ports = defaultdict(list)
for mp_id, port_id in rows:
    polygon_to_ports[mp_id].append(port_id)

# Optional: if we have coordinates table for ports (port_id, lat, lon)
ports_geo = {}
try:
    cur.execute("SELECT port_id, lat, lon FROM ports")
    for pid, plat, plon in cur.fetchall():
        ports_geo[pid] = (plat, plon)
except Exception:
    pass

conn.close()



In [5]:
# Build figure
fig = go.Figure()

# Color maps
courier_colors = px.colors.qualitative.Alphabet * 10
polygon_colors = px.colors.qualitative.Light24 * 10

# Scatter for orders (by polygon color)
# Collect points by polygon id
mp_to_points = defaultdict(lambda: {"lat": [], "lon": []})
for route in routes:
    for oid in route["route"][1:-1]:  # skip warehouse at ends
        lat, lon, mp = order_to_geo.get(oid, (None, None, None))
        if lat is None:
            continue
        mp_to_points[mp]["lat"].append(lat)
        mp_to_points[mp]["lon"].append(lon)

for i, (mp, pts) in enumerate(mp_to_points.items()):
    fig.add_trace(go.Scattergl(
        x=pts["lon"], y=pts["lat"],
        mode="markers",
        marker=dict(size=order_marker_size, color=polygon_colors[i % len(polygon_colors)]),
        showlegend=False
    ))

# Lines per courier (polyline between consecutive orders in its route)
for idx, route in enumerate(routes):
    color = courier_colors[idx % len(courier_colors)]
    coords = []
    for oid in route["route"]:
        lat, lon, _ = order_to_geo.get(oid, (None, None, None))
        if lat is None:
            continue
        coords.append((lat, lon))
    if len(coords) >= 2:
        fig.add_trace(go.Scattergl(
            x=[c[1] for c in coords], y=[c[0] for c in coords],
            mode="lines",
            line=dict(color=color, width=1),
            opacity=line_opacity,
            showlegend=False
        ))

# Optional: draw ports as squares
if show_ports:
    port_lats = []
    port_lons = []
    for mp, ports in polygon_to_ports.items():
        for pid in ports:
            if pid in order_to_geo:
                plat, plon, _ = order_to_geo[pid]
            elif pid in ports_geo:
                plat, plon = ports_geo[pid]
            else:
                continue
            port_lats.append(plat)
            port_lons.append(plon)
    if port_lats:
        fig.add_trace(go.Scattergl(
            x=port_lons, y=port_lats,
            mode="markers",
            marker=dict(size=port_marker_size, symbol="square", color="#000000"),
            showlegend=False
        ))

fig.update_layout(
    width=1200, height=800,
    margin=dict(l=10, r=10, t=10, b=10),
    xaxis_title="Longitude", yaxis_title="Latitude",
    template="plotly_white"
)
fig.show()
